https://rangelands.app/

https://rangelands.app/rap/?biomass_t=herbaceous&ll=38.4262,-123.4373&z=4&landcover_o=0.53&landcover_t=afgc&landcover_y=1987&year=2015
    
https://rangelands.app/rap/

In [ ]:
#! pip install --user rioxarray

In [ ]:
import rasterio
import xarray as xr
import hvplot.xarray
import geoviews as gv
import rioxarray as rx

In [ ]:
import fsspec
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

# return all the COGS for a given year
def get_tifs(last_prefix):
    files = fs.ls(f'eco-w1/in1/{last_prefix}')
    return [f for f in files if f.endswith('tif')]


tif_list = get_tifs('rapv2_cog')

tif_list[0]



In [ ]:
import rasterio
import xarray as xr

#returns an xarray DA
def overview_ugly_xr_read(s3_tif_object):
    da = xr.open_rasterio(f's3://{s3_tif_object}')
    src = rasterio.open(f's3://{s3_tif_object}', 'r')
    #nsub = 4  # original scale factor
    #nsub = 64
    nsub=128
    nsub_half = int(nsub/2)
    data = src.read(out_shape=(1, int(src.height / nsub), int(src.width / nsub)))
    DA = xr.DataArray(data, dims=da.dims, coords=[da.band, da.y[nsub_half:-nsub_half:nsub], da.x[nsub_half:-nsub_half:nsub]])
    DA.attrs = da.attrs
    return(DA)

In [ ]:
cog_url = tif_list[0]
cog_url

In [ ]:
src = rasterio.open(f's3://{cog_url}', 'r')
[src.overviews(i) for i in src.indexes]

In [ ]:
overview=6
print(src.overviews(1)[overview])

In [ ]:
def overview_pretty_xr_read(s3_tif_object):
    da = rx.open_rasterio(f's3://{s3_tif_object}', masked=True, overview_level=6)
   
    return(da)

In [ ]:
years = range(1984,2012)
years

In [ ]:
%%time
howmany = len(years)
dalist =[overview_pretty_xr_read(tif) for tif in tif_list[:howmany]]

da = xr.concat(dalist, dim='band')
da = da.rename({'band':'year'})

In [ ]:
da = da.assign_coords(year=years)

In [ ]:
#da.dims
da.coords

In [ ]:
import hvplot.xarray

img_figure = da.hvplot.image(dynamic=False,rasterize=True,grid=True, clim=(0,50),
                             invert=False, width=650, height=500, cmap='OrRd')

In [ ]:
img_figure.hist()

In [ ]:
da.dims

In [ ]:
da